<h1>Audio</h1>

Wieso sind Töne und Geräusche wichtig?<br>
Auch diese können wie normale Daten wie Text oder Bilder mit einem neuralen Netz verarbeitet werden. Neurale Netze sind dafür sehr gut geeignet, diese können verschiedenste Features erfassen anders als die einfacheren Machine Learning Konzepte. Besonders wenn es komplexere Zusammenhänge gibt wie bei Bildern.

Es gibt unterschiedlichste Use-Cases wie:
- Spracherkennung (Wörter, Befehle, Audio-to-text)
- Klassifizierung von Tönen wie Musik.
- Umgebungswahrnehmung
- Geräusche differenzieren und Unterschiede erkennen

Um den Anfang zu machen, wollen wir als Ziel ein Model trainieren, dass Wörter erkennen soll.:
- Ein ASR Model (Automatic-speech-recognition)

Als Dataset nutzen wir:
> Speech Commands dataset (Warden, 2018) <br>
> https://www.tensorflow.org/datasets/catalog/speech_commands [Letzter Zugriff: 12.08.2024]

Dieses Dataset enthält folgende Wörter im Audioformat: "down", "go", "left", "no", "right", "stop", "up" und "yes"

In [1]:
# Imports 
import os
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf

import IPython
from IPython import display

In [2]:
tf.__version__

'2.17.0'

Dan laden wir das Dataset herunter.

In [2]:
DATASET_PATH = 'data/mini_speech_commands'

data_dir = pathlib.Path(DATASET_PATH)
if not data_dir.exists():
  tf.keras.utils.get_file(
      'mini_speech_commands.zip',
      origin="http://storage.googleapis.com/download.tensorflow.org/data/mini_speech_commands.zip",
      extract=True,
      cache_dir='.', cache_subdir='data')

In [3]:
# Dataset enthält Ordner mit Audiodateien.
commands = np.array(tf.io.gfile.listdir(str(data_dir)))
commands = commands[(commands != 'README.md') & (commands != '.DS_Store')]
print(f'Kommandos: {commands}')

Kommandos: ['down' 'go' 'left' 'no' 'right' 'stop' 'up' 'yes']


Jeder dieser Ordner enthält .wav Audiodateien, die wir abspielen können.
- Durch solch eine Ordnerstruktur kann direkt mit TensorFlow ein Dataset erstellt werden. Siehe TF Datasets.

<h2>Spiele Audiodatei ab</h2>

In [9]:
# Erstelle Liste mit Dateinamen.
files_names_down = os.listdir(os.path.join(DATASET_PATH, "down"))
len(files_names_down)

1000

In [8]:
files_names_down[0]

'004ae714_nohash_0.wav'

Audiodateien könne im Notebook über IPython oder HTML einfach abgespielt werden.

In [10]:
IPython.display.Audio("data/mini_speech_commands/down/004ae714_nohash_0.wav")

In [17]:
# Spiele verschiedene Dateien ab #
# Down # 
rdm = np.random.randint(1, len(files_names_down))
audo_file = DATASET_PATH+"/down/"
print("File: Down")
IPython.display.Audio(audo_file+files_names_down[rdm])

File: Down


In [30]:
# Spiele verschiedene Dateien ab #

categ = commands[np.random.randint(1, len(commands))]
index = np.random.randint(1, len(files_names_down))

categ_path     = DATASET_PATH+f"/{categ}"
list_categ_dir = os.listdir(categ_path)

audo_file = categ_path+f"/{list_categ_dir[index]}"

print(f"File: {audo_file}\nCommando: {categ}")
IPython.display.Audio(audo_file)

File: data/mini_speech_commands/yes/161fcca8_nohash_0.wav
Commando: yes


<h2>Erstelle Dataset</h2>

Mit TensorFlow kann aus einem Verzeichnis direkt ein Dataset abgeleitet werden. Das vereinfach den Umgang mit den Daten.
- Siehe TF Datasets.

In [31]:
# Erstelle Dataset # 
train_ds, val_ds = tf.keras.utils.audio_dataset_from_directory(
    directory=data_dir,
    batch_size=32,
    validation_split=0.2,
    seed=0,
    output_sequence_length=16000,
    subset='both')
train_ds

Found 8000 files belonging to 8 classes.
Using 6400 files for training.
Using 1600 files for validation.


<_BatchDataset element_spec=(TensorSpec(shape=(None, 16000, None), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [36]:
dir(train_ds)

['_GeneratorState',
 '__abstractmethods__',
 '__bool__',
 '__class__',
 '__class_getitem__',
 '__debug_string__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__tf_tracing_type__',
 '__weakref__',
 '_abc_impl',
 '_add_trackable_child',
 '_add_variable_with_custom_getter',
 '_apply_debug_options',
 '_as_serialized_graph',
 '_batch_size',
 '_checkpoint_adapter',
 '_checkpoint_dependencies',
 '_common_args',
 '_consumers',
 '_convert_variables_to_tensors',
 '_copy_trackable_to_cpu',
 '_deferred_dependencies',
 '_deserialization_dependencies',
 '_deserialize_from_proto',
 '_drop_remainder',
 '_export_to_saved_model_graph',
 '_flat_shapes',
 '_f

In [37]:
train_ds.class_names

['down', 'go', 'left', 'no', 'right', 'stop', 'up', 'yes']

In [38]:
# Erstelle Test- und Validationset #
test_ds = val_ds.shard(num_shards=2, index=0)
val_ds  = val_ds.shard(num_shards=2, index=1)

<h2>Visualisierung und weiteres zu Audioformaten</h2>